# Multi-Agent Demo: LangGraph + Recursive Companion

## Simple, Modular Design

### 🎯 The RC Difference: Agents are Just Callables

Unlike LangChain's complex abstractions (Chains, Agents, Tools, Memory), Recursive Companion agents are **simple Python callables**:

```python
# Create an agent
agent = MarketingCompanion()

# Use it anywhere - no special interfaces needed!
result = agent("Why did sales drop?")           # Direct call
result = agent.loop("Why did sales drop?")      # Explicit method
node = RunnableLambda(agent)                    # LangGraph integration
```

No abstract base classes. No framework lock-in. Just modular components that work everywhere.

### What You'll See in This Demo:

This notebook demonstrates how **LangGraph** (workflow orchestration) and **Recursive Companion** (thinking transparency) work together to provide complete observability in multi-agent systems.

1. **LangGraph's Strengths**: 
   - Parallel agent execution
   - Clean state management  
   - Workflow debugging with `print_mode="debug"`
   
2. **RC's Addition**:
   - Full critique/revision history for each agent
   - Convergence metrics and reasoning evolution
   - Deep introspection unavailable through orchestration alone

### Key Insight:
LangGraph tells you **what** happened in your workflow. Recursive Companion shows you **why** each agent reached its conclusions. Together, you get complete system understanding.

See [LangGraph Comparison](https://github.com/hankbesser/recursive-companion/blob/main/docs/LangGraph_Compliment.md) for detailed analysis.

In [ ]:
# SPDX-License-Identifier: MIT
#
# Copyright (c) [2025] [Henry Besser]
#
# This software is licensed under the MIT License.
# See the LICENSE file in the project root for the full license text.

# demos/multi_agent_langgraph_demo.ipynb

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
api_key_status = "Loaded" if os.getenv("OPENAI_API_KEY") else "NOT FOUND - Check your .env file and environment."
print(f"OpenAI API Key status: {api_key_status}")

In [ ]:
# imports
from IPython.display import Image, display, Markdown
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph
from typing import TypedDict
from recursive_companion.base import MarketingCompanion, BugTriageCompanion, StrategyCompanion

In [ ]:
# different models for differents domains
llm_fast  = "gpt-4o-mini"
llm_deep  = "gpt-4.1-mini" 

In [ ]:
# create the agents
# tip read the doctring by hovering over the class
mkt   = MarketingCompanion(llm=llm_fast, temperature=0.8, max_loops=3, similarity_threshold=0.96)
eng   = BugTriageCompanion(llm=llm_deep, temperature=0.3)
plan = StrategyCompanion(llm=llm_fast)

In [ ]:
# RC companions work as drop-in LangGraph nodes!
# The __call__ method makes them compatible with RunnableLambda
# No special integration needed - they just work together

mkt_node  = RunnableLambda(mkt)          # Marketing companion → LangGraph node
eng_node  = RunnableLambda(eng)          # Engineering companion → LangGraph node

# Now these nodes have BOTH:
# - LangGraph's orchestration capabilities (streaming, retries, etc.)
# - RC's thinking transparency (critique/revision history)

In [ ]:
# merge-lambda joins text views into one string
# note: LangGraph passes the entire upstream-state dict to a node.
# with out this function, two upstream nodes are piped straight into strategy, 
# so plan_node will receive a Python dict like {"engineering": "...", "marketing": "..."}.
# That's fine if your StrategyCompanion prompt expects that JSON blob, 
# but most of the time you'll want to concatenate the two strings first.

merge_node = RunnableLambda(
    lambda d: f"### Marketing\n{d['marketing']}\n\n### Engineering\n{d['engineering']}"
)
plan_node  = RunnableLambda(plan)

# Define the state schema for LangGraph
class GraphState(TypedDict):
    input: str
    marketing: str
    engineering: str
    merged: str
    final_plan: str

# Inline LangGraph example (fan-in)
# No extra prompts, no schema gymnastics: simply passing text between the callables the classes already expose.
graph = StateGraph(GraphState)
graph.add_node("marketing_agent",    lambda state: {"marketing": mkt_node.invoke(state["input"])})
graph.add_node("engineering_agent",  lambda state: {"engineering": eng_node.invoke(state["input"])})
graph.add_node("merge_agent",        lambda state: {"merged": merge_node.invoke(state)})
graph.add_node("strategy_agent",     lambda state: {"final_plan": plan_node.invoke(state["merged"])})

graph.add_edge("marketing_agent", "merge_agent")
graph.add_edge("engineering_agent", "merge_agent")
graph.add_edge("merge_agent", "strategy_agent")

graph.add_edge("__start__", "marketing_agent")
graph.add_edge("__start__", "engineering_agent")
graph.set_finish_point("strategy_agent")
workflow = graph.compile()

In [ ]:
#display the graph
display(Image(workflow.get_graph().draw_mermaid_png()))

In [ ]:
# Capture the debug stream output
debug_chunks = []

for chunk in workflow.stream(
    {"input": "App ratings fell to 3.2★ and uploads crash on iOS 17.2. Diagnose & propose next steps."},
    print_mode="debug"
):
    debug_chunks.append(chunk)  # Save each chunk
    print(chunk)
    print("\n")

### 🎯 Extracting Just the Final Strategy Result

Now let's extract ONLY the final strategy agent's conclusion from all that debug output:

In [ ]:
# Extract the final strategy result from the debug stream
# The strategy agent's result is in the last task_result with name='strategy_agent'
strategy_result = None
for chunk in reversed(debug_chunks):  # Search from the end
    if (chunk.get('type') == 'task_result' and 
        chunk.get('payload', {}).get('name') == 'strategy_agent'):
        # The result is in payload.result[0][1] 
        # (list of tuples where first item is key, second is value)
        strategy_result = chunk['payload']['result'][0][1]
        break

print("\n" + "="*80)
print("📋 EXTRACTED STRATEGY RESULT:")
print("="*80)
if strategy_result:
    print(strategy_result[:500] + "..." if len(strategy_result) > 500 else strategy_result)

### 📊 Capturing LangGraph's Debug Stream

First, we need to capture all the debug output to extract results later:

In [ ]:
# === OBSERVABILITY COMPARISON ===
print("=" * 80)
print("🔍 COMPLEMENTARY OBSERVABILITY")
print("=" * 80)

print("\n✅ LangGraph's Workflow Debugging (print_mode='debug'):")
print("  • Task scheduling and execution order")
print("  • Node inputs/outputs and state transitions")  
print("  • Parallel execution timing")
print("  • Error handling and retries")
print("  → Perfect for debugging orchestration issues!")

print("\n✅ Recursive Companion's Thinking Transparency:")
print("  • Draft → Critique → Revision cycles")
print("  • Similarity scores and convergence patterns")
print("  • Complete reasoning audit trail")
print("  • Why agents reached specific conclusions")
print("  → Essential for understanding agent reasoning!")

print("\n🎯 Better Together:")
print("  • LangGraph: See the workflow execution flow")
print("  • RC: Understand the thinking behind decisions")
print("  • Zero integration overhead - RC companions are drop-in nodes")
print("\n👉 Scroll down to see both types of observability in action!")

In [ ]:
# Extract all agent results from the debug stream
agent_results = {}

for chunk in debug_chunks:
    if chunk.get('type') == 'task_result' and chunk.get('payload'):
        agent_name = chunk['payload'].get('name', '').replace('_agent', '')
        if chunk['payload'].get('result'):
            # Result is in format: [(key, value)]
            result_value = chunk['payload']['result'][0][1]
            agent_results[agent_name] = result_value

print("📊 ALL AGENT RESULTS FROM DEBUG STREAM:")
print("="*60)
for agent, result in agent_results.items():
    print(f"\n{agent.upper()}:")
    print(result[:200] + "..." if len(result) > 200 else result)
    print("-"*60)

### 📈 Extracting ALL Agent Results 

Want to see what EVERY agent said? Let's extract all results from the debug stream:

## 🤔 Notice How Complex Extracting Results from LangGraph Is?

### LangGraph Debug Stream Extraction (Complex!)
- Need to capture all chunks in a list
- Parse nested dictionary structure (`chunk['payload']['result'][0][1]`)
- Search through chunks to find the right `task_result`
- No access to iteration details or thinking process

### RC Access Pattern (Simple!)
```python
# With RC, everything is immediately accessible:
mkt.run_log           # Full iteration history
mkt.run_log[-1]       # Last iteration details
mkt.transcript_as_markdown()  # Formatted thinking process

# No parsing, no searching, no nested dictionaries!
```

This complexity difference becomes even more pronounced when you need to debug WHY an agent made a decision...

In [ ]:
result = workflow.invoke(
    {"input": "App ratings fell to 3.2★ and uploads crash on iOS 17.2. Diagnose & propose next steps."}
)

In [ ]:
final = result.get("final_plan", "")

In [ ]:
print("\n=== FINAL PLAN ===\n")
display(Markdown(final))

In [ ]:
# === After LangGraph workflow completes ===
print("\n🔍 DEEP INTROSPECTION - What LangGraph CAN'T normally show you:\n")
# Show iteration counts
print(f"Marketing iterations: {len(mkt.run_log)}")
print(f"Engineering iterations: {len(eng.run_log)}")
print(f"Strategy iterations: {len(plan.run_log)}")
# Show why each converged
print("\n📊 CONVERGENCE ANALYSIS:")
for name, agent in [("Marketing", mkt), ("Engineering", eng), ("Strategy", plan)]:
    if len(agent.run_log) < agent.max_loops:
        print(f"{name}: Converged early (quality threshold reached)")
    else:
        print(f"{name}: Used all {agent.max_loops} iterations")


In [ ]:
# Show full thinking process - BEAUTIFULLY FORMATTED!
# No parsing, no JSON manipulation - just readable markdown
print("\n🧠 MARKETING THINKING PROCESS:")
display(Markdown(mkt.transcript_as_markdown()))  # Ready for reports, logs, or UI!

## 📝 Beautiful Output with Zero Parsing!

Notice how RC's `transcript_as_markdown()` gives you perfectly formatted output:
- **No JSON parsing needed** (unlike LangGraph's nested dictionaries)
- **Ready for reports, logs, or UI display**
- **Each iteration clearly separated** with Draft → Critique → Revision
- **Just one method call** vs complex chunk extraction

Compare this to extracting from LangGraph's debug stream above - night and day!

In [ ]:
# Engineering thinking process - each iteration clearly separated
print("\n🔧 ENGINEERING THINKING PROCESS:")
display(Markdown(eng.transcript_as_markdown()))  # Draft → Critique → Revision for each loop

In [ ]:
# Strategy synthesis - complete thinking audit trail
print("\n🎯 STRATEGY SYNTHESIS PROCESS:")
display(Markdown(plan.transcript_as_markdown()))  # One method = full thinking history!